In [189]:
from matching.games import HospitalResident
import random;
def match(choices, capacities):
    players = []
    hospital_preferences = dict()
    assignments = dict()
    #fill the assignments and hospital pref with empty lists
    for (h, c) in capacities.items():
        assignments[h] = []
        hospital_preferences[h] = []
    #add all players that wanted an hospital to that hospital's preference list
    for (p, c) in choices.items():
        for h in c:
            hospital_preferences[h].append(p)
    #shuffle the hospital preference list
    for (h, p) in hospital_preferences.items():
            random.shuffle(hospital_preferences[h])
    #while there are still things waiting to be assigned
    cont = True
    while cont:
        cont = False
        #create a matching
        game = HospitalResident.create_from_dictionaries(choices, hospital_preferences, capacities)
        matching = game.solve(optimal="resident")
        #process it
        for (h, ass) in matching.items():
            h = str(h)
            #add it to the assignments
            assignments[h] += ass
            #substract the capacities
            capacities[h] -= len(ass)
            #prevent a player from getting assigned to the same hospital twice
            for p in ass:
                p = str(p)
                hospital_preferences[h].remove(p)
                choices[p].remove(h)
        #shuffle the hospital preferences and see if it should continue
        for (h, p) in hospital_preferences.items():
            random.shuffle(hospital_preferences[h])
            if capacities[h] > 0 and len(p) > 0:
                cont = True
        #cleanup : delete hospital and players that are finished
        cleanup = True
        while cleanup: 
            cleanup = False
            to_remove = []
            for (h, p) in hospital_preferences.items():
                #if an hospital is full or noone wants to go there, delete it
                if not (capacities[h] > 0 and len(p) > 0):
                    to_remove.append(h)
                    cleanup = True
                    for (p, hl) in choices.items():
                        if h in hl:
                            hl.remove(h)
            for h in to_remove:
                del hospital_preferences[h]
            
            #if a player don't want anything anymore, delete it
            to_remove = []
            for (p, hl) in choices.items():
                if len(hl) == 0:
                    to_remove.append(p)
                    cleanup = True
                    for (h, pl) in hospital_preferences.items():
                        if p in pl:
                            pl.remove(p)
            for p in to_remove:
                del choices[p]   
    return assignments

In [192]:
choices = {"j1":["a1","a2","a5"], "j2":["a2","a3"], "j3":["a4","a5"], "j4":["a5","a6"], "j5":["a3","a1"], "j6":["a4","a5"]}
cap = {"a1":1,"a2":2,"a3":1,"a4":4,"a5":5,"a6":2,}
print(match(choices, cap))

{'a1': [j1], 'a2': [j2, j1], 'a3': [j5], 'a4': [j3, j6], 'a5': [j4, j6, j3, j1], 'a6': [j4]}
